In [3]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#
# 
#
# Script to download and visualize the precipitation data from NCDC, NOAA
#
#         
#
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cdo_api_py import Client



In [16]:

# Define a function to create folders
def ResultsFolder(Folder):
    if not os.path.exists(Folder):
        os.makedirs(Folder)

# Define a function to delete empty folders
def delete_empty(path):
    for folder_name in os.listdir(path):
        folder_path = os.path.join(path, folder_name)
        if os.path.isdir(folder_path):
            try:
                os.rmdir(folder_path)
                print(f"* Deleted empty folder: {folder_path}")
            except OSError:
                # The folder is not empty or there is no permission to delete it
                pass
# Define a function to get precipitation data
def GetRain(USGS_Gauge, start_date, end_date, extent,Folder1,Folder2):
    my_client = Client(my_token, default_units="standard", default_limit=1000)
    datasetid = "PRECIP_15"
   # Folder1 = f'C:/Users/mgalib/Box/Finley_UH_work/Data/TN/Precipitation_data/Raw_Rainfall_Data_{USGS_Gauge}_{start_date}/'
    ResultsFolder(Folder1)

    try:
        stations = my_client.find_stations(datasetid=datasetid,
                                           extent=extent,
                                           startdate=pd.to_datetime(start_date),
                                           enddate=pd.to_datetime(end_date),
                                           return_dataframe=True)
        if stations.empty:
            raise ValueError("No stations found in the initial extent.")
    except Exception as e:
        print(f"No stations found for {USGS_Gauge} within the given extent: {extent}")
        print("Attempting to expand the extent by 0.2 degrees...")
        for key in ['north', 'south', 'east', 'west']:
            if 'north' == key or 'east' == key:
                extent[key] += 0.2
            else:
                extent[key] -= 0.2
        stations = my_client.find_stations(datasetid=datasetid,
                                           extent=extent,
                                           startdate=pd.to_datetime(start_date),
                                           enddate=pd.to_datetime(end_date),
                                           return_dataframe=True)
        if stations.empty:
            print("No stations found even after expanding the extent. Skipping this gauge.")
            return  # Exiting the function if no stations are found after extent adjustment


    rainfall = []
    for stationid in stations.id:
        station_id = stationid.split(":")[1]
        csv_path = Folder1 + datasetid + "min_COOP_" + station_id + ".csv"
        try:
            raw_rainfall = my_client.get_data_by_station(datasetid=datasetid, stationid=stationid, startdate=pd.to_datetime(start_date), enddate=pd.to_datetime(end_date), return_dataframe=True)
            raw_rainfall.to_csv(csv_path)
            df = pd.read_csv(csv_path)
            df = df[['date', 'QPCP']]
            rainfall.append(df)
        except Exception as msg:
            print(f"Warnings for {USGS_Gauge}: {msg}")
            continue

    if not rainfall:
        print(f"No valid data collected for {USGS_Gauge}. Exiting process.")
        return

    rainfall = pd.concat(rainfall, axis=0, ignore_index=True)
    rainfall.to_csv(Folder1 + 'rainfall of all stations.csv')
    rainfall.set_index('date', inplace=True)
    rainfall.replace(999.99, pd.NA, inplace=True)
    rainfall.dropna(inplace=True)

    rainfall_mean = pd.DataFrame({'rainfall_in': rainfall['QPCP'].groupby(rainfall.index).mean()})
    rainfall_mean.index = pd.to_datetime(rainfall_mean.index)
    # Check if the rainfall_mean DataFrame is empty
    if rainfall_mean.empty:
        print(f"No valid mean rainfall data for {USGS_Gauge}. Exiting process.")
        return

    #Folder2 = f'C:/Users/mgalib/Box/Finley_UH_work/Data/TN/TN_2010/Hyetograph/Results_Rainfall_{USGS_Gauge}_{start_date}/'
    ResultsFolder(Folder2)
    rainfall_mean.to_csv(Folder2 + 'rainfall_mean.csv')

    plt.figure(figsize=(20, 18))
    rainfall_mean.plot.bar()
    plt.title("Hyetograph")
    plt.legend(["Rainfall"], loc='best', edgecolor='k')
    plt.xlabel("Date")
    plt.ylabel("Rainfall (in)")
    plt.savefig(Folder2 + "Hyetograph.jpeg")
    plt.close()

    plt.figure(figsize=(20, 10))
    rainfall_mean.plot(style='bo')
    plt.title("Rainfall scatter")
    plt.legend(["Rainfall"], loc='best', edgecolor='k')
    plt.xlabel("Date")
    plt.ylabel("Rainfall (in)")
    plt.savefig(Folder2 + "Rainfall_scatter.jpeg")
    plt.close()


In [17]:
## Main program

# access the data from the NCDC database with my token

my_token = 'KFeRXLcZnMpQdBsNWwOPtvBBMOLkmAcW'
watershed_extent = "C:/Users/mgalib/Box/Finley_UH_work/Data/TN/watershed_extent_TN.csv"
NSEW = pd.read_csv(watershed_extent,dtype={'Gauge_No':str})

for i in range(len(NSEW)):
    # The extend is the lat, long of the target region.
    extent = dict()
    Directions = ['north','south','east','west']
    USGS_Gauge = str(NSEW.iloc[i,0])

    for j in range(4):
        extent[Directions[j]] = NSEW.iloc[i,j+1]
    
    start_date = pd.to_datetime(NSEW.iloc[i, 5]).strftime('%Y-%m-%d')
    end_date = pd.to_datetime(NSEW.iloc[i, 6]).strftime('%Y-%m-%d')
    Folder1 = f'C:/Users/mgalib/Box/Finley_UH_work/Data/TN/Precipitation_data/Raw_Rainfall_Data_{USGS_Gauge}_{start_date}/'
    Folder2 = f'C:/Users/mgalib/Box/Finley_UH_work/Data/TN/Hyetograph/Results_Rainfall_{USGS_Gauge}_{start_date}/'

  
    GetRain(USGS_Gauge,start_date,end_date,extent,Folder1,Folder2)
    print("Rainfall data processing is done for " + USGS_Gauge + " starting from " + start_date)


delete_empty('C:/Users/mgalib/Box/Finley_UH_work/Data/TN/Precipitation_data')

https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?datasetid=PRECIP_15&startdate=2018-05-25&enddate=2018-06-10&extent=36.47054977&extent=-85.87185917&extent=36.56569568&extent=-85.80165628&limit=1000&units=standard
No stations found for 3312259 within the given extent: {'north': 36.56569568, 'south': 36.47054977, 'east': -85.80165628, 'west': -85.87185917}
Attempting to expand the extent by 0.2 degrees...
https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?datasetid=PRECIP_15&startdate=2018-05-25&enddate=2018-06-10&extent=36.270549769999995&extent=-86.07185917&extent=36.76569568&extent=-85.60165628&limit=1000&units=standard
No stations found even after expanding the extent. Skipping this gauge.
Rainfall data processing is done for 3312259 starting from 2018-05-25
https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?datasetid=PRECIP_15&startdate=2009-04-12&enddate=2009-04-28&extent=36.31355449&extent=-86.79300794&extent=36.41892638&extent=-86.71621875&limit=1000&units=standard
No stations fou

c:\temp\envs\purdue\Lib\site-packages\pandas\plotting\_matplotlib\core.py:580: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = self.plt.figure(figsize=self.figsize)


Rainfall data processing is done for 3430147 starting from 2004-04-15
https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?datasetid=PRECIP_15&startdate=2015-04-01&enddate=2015-04-14&extent=36.06495557&extent=-86.81620105&extent=36.16118001&extent=-86.72033451&limit=1000&units=standard
No stations found for 3431300 within the given extent: {'north': 36.16118001, 'south': 36.06495557, 'east': -86.72033451, 'west': -86.81620105}
Attempting to expand the extent by 0.2 degrees...
https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?datasetid=PRECIP_15&startdate=2015-04-01&enddate=2015-04-14&extent=35.86495557&extent=-87.01620105&extent=36.361180010000005&extent=-86.52033451&limit=1000&units=standard
No stations found even after expanding the extent. Skipping this gauge.
Rainfall data processing is done for 3431300 starting from 2015-04-01
https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?datasetid=PRECIP_15&startdate=2016-04-25&enddate=2016-05-11&extent=36.06439975&extent=-86.91755364&extent=36.

C:\Users\mgalib\AppData\Local\Temp\ipykernel_26288\3302117483.py:85: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(20, 18))


Rainfall data processing is done for 3435970 starting from 2004-05-24
https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?datasetid=PRECIP_15&startdate=2017-03-24&enddate=2017-04-09&extent=35.58924747&extent=-83.58869221&extent=35.70895269&extent=-83.38886094&limit=1000&units=standard
No stations found for 3469251 within the given extent: {'north': 35.70895269, 'south': 35.58924747, 'east': -83.38886094, 'west': -83.58869221}
Attempting to expand the extent by 0.2 degrees...
https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?datasetid=PRECIP_15&startdate=2017-03-24&enddate=2017-04-09&extent=35.389247469999994&extent=-83.78869221000001&extent=35.90895269&extent=-83.18886094&limit=1000&units=standard
No stations found even after expanding the extent. Skipping this gauge.
Rainfall data processing is done for 3469251 starting from 2017-03-24
https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?datasetid=PRECIP_15&startdate=2019-06-30&enddate=2019-07-16&extent=36.37321371&extent=-83.05688717&exte

...

<Figure size 2000x1000 with 0 Axes>

<Figure size 2000x1800 with 0 Axes>

<Figure size 2000x1000 with 0 Axes>

<Figure size 2000x1800 with 0 Axes>

<Figure size 2000x1000 with 0 Axes>